# Metodologia general creación DataSet CSV

* 1. clasificar con modelo enetrenado imagenes-
* * 1.1 Procesar raster, calculo de indices y deltas
* * 1.2 Clasificar raster según modelo seleccionado previamente
* 2. creación de CSV
* * 2.1 Crear CSV por cada Zona
* * 2.2 Unificar todos los csv de una zona 


## Observaciones generales

* 1. Las imagenes a analizar contienen por nombre pre+num+.tif y post+num+.tif
* 2. Se asume que cada imagen pre tiene una post, es importante el nombre del rater se encuentre bien
* 3. Los archivos csv se guardarán en las mismas direcciones, las estadisticas generales en la carpeta principal
* 4. Carpetas y objetos obligatorios
* * Funciones -> confunciones en su interior
* * Modelos - base  -> con el modelo base obtenido de experimento anterior
* * imagenesAnalisis - 'nombre zona' -> en su interior con imagen pre y post, como indica en 1
* * notebook con archivos de ejecución y prueba

### Cargar librerias

In [1]:
# -*- coding: utf-8 -*-
import os,sys

#consegir direccion base de experimentos
base = os.getcwd().strip(' ').split('/')
dirBase=''
for i in range(len(base)-1):
    dirBase+=base[i]+'/'
    
sys.path.append(dirBase+'Funciones')
print('La direccion base es: '+dirBase)

import indices as ind
import imagenes as im
import toolbox as tl
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns #Seaborn is a Python data visualization library based on matplotlib
from time import time
import pandas as pd


La direccion base es: /home/pc/Tesis MCC/


### Direcciones de carpetas y archivos

In [2]:
#Direccion general

dirImAnalisis= dirBase+'imagenesAnalisis/'

modelBase=dirBase+'modelos/base/modelo_ELM_delRasInd.sav'  #modelo base entrenado con zonas visuales
#nombre de imagen según modelo
rasClassModel='dIm+dInd.tif'
rasClasificado='rasterClasificado.tif'
shpClasificado='shpClasificado.shp'
csvClasificado='CsvClasificado.csv'

#1. clasificar con modelo entrenado imagenes
#zonas de experimento, se encuentra dentro de la carpete de imagenes a analizar
zonas=['maule','concepcion','temuco']


# ***************   Inicio experimento     ***************

## 1. clasificar con modelo entrenado imagenes

### 1.1. Procesar Raster e indices

In [6]:
print('Creación de Raster procesados')
for zona in zonas:
    dirZona=dirImAnalisis+zona+'/'
    rasProcess=dirZona+'procesadas/'
    tl.crearCarpeta(rasProcess)
    contador=1
    
    while os.path.isfile(dirZona+'pre'+str(contador)+'.tif'): #si existe la imagen pre+num (contador) entonces
        dPre=dirZona+'pre'+str(contador)+'.tif'
        dPost=dirZona+'post'+str(contador)+'.tif'
        pre=(im.raster(dPre,'bandasSen2cor'))
        post=(im.raster(dPost,'bandasSen2cor'))
        dirProcesaZona=rasProcess+'/'+str(contador)+'/'#dirección de raster procesado en esa zona
        tl.crearCarpeta(dirProcesaZona)
        
        print('Procesando en: '+dirZona+'pre'+str(contador)+'.tif')

        #cargar y procesar imagenes pre y post, calculo de indices
        #	MIRBI
        #	NBR2
        #	BAI
        #	BAIS2

        #cargar y procesar imagnes pre y post
        indPre=ind.indices1(pre)
        indPost=ind.indices1(post)

        #Calculos sobre imagenes previas + indices
        previas=[]
        previas.append(pre.listaBandas)#lista con valores de todas las bandas
        previas.append(indPre)
        ImaPre=ind.unir(previas)

        tl.guardarRaster(ImaPre,pre,dirProcesaZona+'pre+ind.tif') #pre junto con indices
        tl.guardarRaster(indPre,pre,dirProcesaZona+'indicesPre.tif') #indices pre

        #Calculos sobre imagenes previas + indices
        posteriores=[]
        posteriores.append(post.listaBandas)
        posteriores.append(indPost)
        ImaPost=ind.unir(posteriores)
        tl.guardarRaster(ImaPost,post,dirProcesaZona+'post+ind.tif')#post junto con indices
        tl.guardarRaster(indPost,post,dirProcesaZona+'indicesPost.tif')#indices post

        #Calcula delta de imagenes pre y post mas indices
        deltasImInd=ind.delta(ImaPre,ImaPost)
        tl.guardarRaster(deltasImInd,post,dirProcesaZona+'dIm+dInd.tif')#delta imagenes e indices

        deltasInd=ind.delta(indPre,indPost)
        tl.guardarRaster(deltasInd,post,dirProcesaZona+'deltasInd.tif')#delta indices

        deltas=ind.delta(pre.listaBandas,post.listaBandas)
        tl.guardarRaster(deltas,post,dirProcesaZona+'deltas.tif') #delta ima
        
        #liberar memoria de apertura de Raster
        del pre
        del post
        #siguiente zona de raster
        contador+=1



Creación de Raster procesados
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/maule/pre1.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/maule/pre2.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/maule/pre3.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/concepcion/pre1.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/concepcion/pre2.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/concepcion/pre3.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/temuco/pre1.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/temuco/pre2.tif
Raster Borrado
Raster Borrado
Procesando en: /home/pc/Tesis MCC/imagenesAnalisis/temuco/pre3.tif
Raster Borrado
Raster Borrado


### 1.2 Clasificar todos los raster procesados con mejor modelo

* Se clasifica con solo un modelo, el mejor de todos los evaluados en etapas anteriores
* Recordar que el modelo es "modelBase"

In [5]:
print('Clasificar Raster según modelo en cada Zona')


for zona in zonas:
    dirRastZona=zona
    dirZona=dirImAnalisis+zona+'/procesadas/'
    rasters=tl.listarSubDirectorios(dirZona)

    for rastZona in rasters: #raster en esa zona

        dRastClassZona=dirZona+rastZona+'/'+rasClassModel#direccion de raster a clasificar

        nomRasClass=dirZona+rastZona+'/'+rasClasificado
        nomShpClass=dirZona+rastZona+'/'+shpClasificado
        print('**********Inicio Proceso clasificación: '+dRastClassZona)
        #print(nomRasClass)
        #print(nomShpClass)
        #print(dRastClassZona)

        rastClass=im.raster(dRastClassZona,'sinEscala') 
        tl.ClasificarPixel(rastClass,modelBase,nomRasClass)
        tl.polygonize(nomRasClass,nomShpClass)

        del rastClass   
        print('**********Final Proceso clasificación: '+dRastClassZona)



Clasificar Raster según modelo en cada Zona
**********Inicio Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/dIm+dInd.tif
/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/rasterClasificado.tif
Raster Borrado
**********Final Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/dIm+dInd.tif
**********Inicio Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/3/dIm+dInd.tif
/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/3/rasterClasificado.tif
Raster Borrado
**********Final Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/3/dIm+dInd.tif
**********Inicio Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/1/dIm+dInd.tif
/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/1/rasterClasificado.tif
Raster Borrado
**********Final Proceso clasificación: /home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/1/dIm+dInd.tif
**********Inicio Proceso clasificac

# 2. creación de CSV

### 2.1 Crear CSV por cada Zona

* Recorrer todas las Zonas y muestras
* Por cada Zona y Muestra se construye un archivo csv 
* * Se debe definir los Raster procesados y Raster Clasificado con mejor modelo previamente.
* * En la exploración de archivos de carpeta, buscar solo los raster y excluir el rasterClasificado
* Los CSV se crean de la siguiente forma:
* * Unificar todos los csv de una zona: unir los csv de todos los experimentos en un mismo csv en raiz de la zona
* * Unificar todos los csv de zonas en un solo archivo: unir los csv de zonas en un mismo csv en raiz de imagenesA..

In [3]:
print('Generar archivos CSV por cada Zona y Muestra')

for zona in zonas: #todas las zonas
    dirRastZona=zona
    dirZona=dirImAnalisis+zona+'/procesadas/'
    
    rasters=tl.listarSubDirectorios(dirZona) #cada Experimento

    for rastZona in rasters: #todos los experimentos
        inicio=time()
        dZona=dirZona+rastZona
        nomRasClass=dZona+'/'+rasClasificado
        
        #consultar por todos los raster existenetes Procesados
        archivos=os.listdir(dZona)# escanerar todos los archivos en la carpeta zona, muestra
        
        for arc in archivos: #eliminar todos los archivos de lista que no contenga .tif
            if arc.find(".tif")==-1:
                archivos.remove(arc)
        archivos.remove(rasClasificado)#eliminar elemento rasterClasificado     
        #print(archivos)
        #Observación: no está eliminando los archivos sin extención .tif, esto genera errores pasados por Try
        
        print('**********Inicio Proceso clasificación en zona: '+zona)
        for arc in archivos:
            dRast=dZona+'/'+arc
            #print(dRast)
            (nombre, sep, ext) = arc.rpartition('.')#extraer solo nombre para crear CsV
            nomCsvClass=dZona+'/'+nombre+csvClasificado

            try:
                im.rasterToCSV(dRast,nomRasClass,nomCsvClass)
            except:
                print("ERROR !!!!!!!!!!!!!!!!")
     
        final=time()-inicio
        print('**********Final Proceso clasificación: '+str(round(final,2))+' s en zona: '+zona)
        
           





Generar archivos CSV por cada Zona y Muestra
**********Inicio Proceso clasificación en zona: maule


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/indicesPostCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/indicesPreCsvClasificado.csv' construido!!
ERROR !!!!!!!!!!!!!!!!



***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/dIm+dIndCsvClasificado.csv' construido!!
ERROR !!!!!!!!!!!!!!!!

ERROR !!!!!!!!!!!!!!!!



***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/deltasCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/pre+indCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/post+indCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/maule/procesadas/2/deltasIndCsvClasifica



***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/indicesPostCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/indicesPreCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/dIm+dIndCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/deltasCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/pre+indCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/post+indCsvClasificado.csv' construido!!


***** CSV de Analisis '/home/pc/Tesis MCC/imagenesAnalisis/temuco/procesadas/1/deltasIndCsvClasificado.csv' construido!!
**********Final Proceso clasificación: 70.44 s en zona: temuco


### 2.2 Unificar todos los csv de una zona

* Constuir un unico data set por zona, con cada muestra
* 1. Encontrar csv de experimento
* 2. unir con otros experimentos en un solo archivo, se debe controlar la apertura de la primera imagen y las siguiente se añaden


In [20]:
print('Unificar archivos CSV de experimentos de cada Zona')
inicio=time() #captura tiempos de ejecución

for zona in zonas: #todas las zonas
    dirRastZona=zona
    dirZona=dirImAnalisis+zona+'/procesadas/'
    direccionZona=dirImAnalisis+zona+'/'
    rasters=tl.listarSubDirectorios(dirZona) #cada Experimento

    #Archivos CSV por cada zona
    archivosCSV=[]
    for rastZona in rasters:
        dZona=dirZona+rastZona
        archivosCSV=os.listdir(dZona)
        archivosCSV=tl.conservarListaPexten(archivosCSV,".csv")
    #print(archivosCSV)
    contador=0 #control ingreso unico a Experimento de zona
    dicCSV={}
    
    
    for rastZona in rasters: #todos los experimentos
        dZona=dirZona+rastZona
        nomRasClass=dZona+'/'+rasClasificado
        
        if contador==0:#primera Zona, abre y carga todas las imagenes en un diccionario
            for arc in archivosCSV:
                dCsv=dZona+'/'+arc
                pandasCSV=pd.read_csv(dCsv,sep=';')
                dicCSV[arc]=pandasCSV #crea Diccionario
                
        elif contador == (len(rasters)-1):#si esta en la ultima zona, cierra y guarda contenido de diccionario
            for arc in archivosCSV:
                dCsv=dZona+'/'+arc
                pandasCSV=pd.read_csv(dCsv,sep=';')
                dicCSV[arc]=pd.concat([dicCSV[arc],pandasCSV],axis=0) #une data set de experimentos en esa zona
                guardarCSV=dirZona+arc
                dicCSV[arc].to_csv(guardarCSV, index=False, encoding='utf-8',sep=';')#guarda
            dicCSV.clear() #elimina contenido de diccionario
        else: # si se encuenta en zonas intermedias, une los csv

            for arc in archivosCSV:
                dCsv=dZona+'/'+arc
                pandasCSV=pd.read_csv(dCsv,sep=';')
                dicCSV[arc]=pd.concat([dicCSV[arc],pandasCSV],axis=0) #une data set de experimentos en esa zona
        contador+=1
    

            


Unificar archivos CSV de experimentos de cada Zona


## 2.2 Unificar todos los csv de zonas en un solo archivo

In [60]:
print('Unificar archivos CSV de experimentos de cada Zona')
inicio=time() #captura tiempos de ejecución

contador=0 #control ingreso unico a zona
dicCSV={}

for zona in zonas: #todas las zonas
    dirZona=dirImAnalisis+zona+'/procesadas/'
    rasters=tl.listarSubDirectorios(dirZona) #cada Experimento

    #Archivos CSV por cada zona
    archivosCSV=[]
    
    
    for rastZona in rasters:
        dZona=dirZona+rastZona
        archivosCSV=os.listdir(dZona)
        archivosCSV=tl.conservarListaPexten(archivosCSV,".csv")
    
    
        
    if contador==0:#primera Zona, abre y carga todas las imagenes en un diccionario
        for arc in archivosCSV:
            dCsv=dirZona+arc
            pandasCSV=pd.read_csv(dCsv,sep=';')
            dicCSV[arc]=pandasCSV #crea Diccionario
                
    elif contador == (len(rasters)-1):#si esta en la ultima zona, cierra y guarda contenido de diccionario
        for arc in archivosCSV:
            dCsv=dirZona+arc
            pandasCSV=pd.read_csv(dCsv,sep=';')
            dicCSV[arc]=pd.concat([dicCSV[arc],pandasCSV],axis=0) #une data set de experimentos en esa zona
            guardarCSV=dirImAnalisis+arc
            dicCSV[arc].to_csv(guardarCSV, index=False, encoding='utf-8',sep=';')#guarda
        dicCSV.clear() #elimina contenido de diccionario
    else: # si se encuenta en zonas intermedias, une los csv

        for arc in archivosCSV:
            dCsv=dirZona+arc
            pandasCSV=pd.read_csv(dCsv,sep=';')
            dicCSV[arc]=pd.concat([dicCSV[arc],pandasCSV],axis=0) #une data set de experimentos en esa zona
            
    contador+=1
   
print('Fin Unificar archivos CSV de experimentos de cada Zona en: '+str(time()-inicio)+' s')


Unificar archivos CSV de experimentos de cada Zona
Fin Unificar archivos CSV de experimentos de cada Zona en: 968.363409280777 s


# se han creado todos los data Set .CSV